In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import math
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split

In [ ]:
#frames, id = [], []
path = "/content/drive/MyDrive/Multimodal Project/finals/CREMAD/videos"
op_path = "/content/drive/MyDrive/Multimodal Project/finals/CREMAD/id_data"
for file_name in os.listdir(path):
    identity = file_name.split("_")[0]
    emotion = file_name.split("_")[2]
    output_path = os.path.join(op_path, identity)
    if not os.path.exists(output_path):
      os.makedirs(output_path)
    video_path = os.path.join(path, file_name)
    cap = cv2.VideoCapture(video_path)
    i = 0
    frame_number = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_number % 5 == 0:
          loc = os.path.join(output_path, f"{emotion}_{i}.jpg")
          cv2.imwrite(loc, frame)
          i += 1
        frame_number += 1
        # id.append(identity)
        # frames.append(np.array(img_to_array(frame) / 255.0))


In [ ]:
path = "/content/drive/MyDrive/Multimodal Project/finals/CREMAD/id_data"
labels = os.listdir(path)
train_df, test_df, val_df = [], [], []
for l in labels:
  imgs = os.listdir(path+"/"+l)
  ni = len(imgs)
  train_size = math.ceil(0.7*ni)
  val_size = math.ceil(0.15*ni)
  test_size = ni - train_size - val_size
  train, test = train_test_split(imgs, test_size = test_size, shuffle = True)
  train, val = train_test_split(train, train_size = train_size, shuffle = True)
  for img in train:
    img_path = path+"/"+l+"/"+img
    train_df.append([str(img_path),l])
  for img in test:
    img_path = path+"/"+l+"/"+img
    test_df.append([str(img_path),l])
  for img in val:
    img_path = path+"/"+l+"/"+img
    val_df.append([str(img_path),l])

train_df = pd.DataFrame(train_df, columns =['path', 'label'])
test_df = pd.DataFrame(test_df, columns =['path', 'label'])
val_df = pd.DataFrame(val_df, columns =['path', 'label'])

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    x_col='path',
                                                    y_col='label',
                                                    batch_size=64,class_mode='sparse')
val_datagen = ImageDataGenerator()
val_generator = val_datagen.flow_from_dataframe(dataframe=val_df,
                                                x_col='path',
                                                y_col='label',
                                                batch_size=64,
                                                shuffle=False,
                                                class_mode='sparse')


test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                                  x_col='path',
                                                  y_col='label',
                                                  batch_size=64,
                                                  shuffle=False,
                                                  class_mode='sparse')

Found 2905 validated image filenames belonging to 91 classes.
Found 661 validated image filenames belonging to 91 classes.
Found 529 validated image filenames belonging to 91 classes.


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

vgg16_model = VGG16(pooling='avg', weights='imagenet', include_top=False, input_shape=(224,224,3))
for layers in vgg16_model.layers:
            layers.trainable=False
last_output = vgg16_model.layers[-1].output
vgg_x = Flatten()(last_output)
vgg_x = Dense(128, activation = 'relu')(vgg_x)
vgg_x=BatchNormalization()(vgg_x)
vgg_x=Dropout(0.2)(vgg_x)
vgg_x = Dense(91, activation = 'softmax')(vgg_x)
vgg16_model = Model(vgg16_model.input, vgg_x)
vgg16_model.compile(loss = 'sparse_categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
history = vgg16_model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=20,
                    callbacks=[early_stopping],
                    verbose=1)
vgg16_model.save_weights('/content/drive/MyDrive/Multimodal Project/finals/vgg16.h5')

# Evaluate the model on training and test sets
train_loss, train_acc = vgg16_model.evaluate(train_generator, verbose=0)

print('Train accuracy:', train_acc)

Epoch 1/20
46/46 [==============================] - 54s 812ms/step - loss: 3.0343 - accuracy: 0.4124 - val_loss: 2.1503 - val_accuracy: 0.5008
Epoch 2/20
46/46 [==============================] - 26s 552ms/step - loss: 1.0201 - accuracy: 0.9384 - val_loss: 0.6331 - val_accuracy: 0.9864
Epoch 3/20
46/46 [==============================] - 25s 550ms/step - loss: 0.3427 - accuracy: 0.9983 - val_loss: 0.2309 - val_accuracy: 1.0000
Epoch 4/20
46/46 [==============================] - 26s 559ms/step - loss: 0.1546 - accuracy: 1.0000 - val_loss: 0.1188 - val_accuracy: 1.0000
Epoch 5/20
46/46 [==============================] - 27s 587ms/step - loss: 0.0861 - accuracy: 0.9997 - val_loss: 0.0726 - val_accuracy: 1.0000
Epoch 6/20
46/46 [==============================] - 27s 578ms/step - loss: 0.0604 - accuracy: 1.0000 - val_loss: 0.0462 - val_accuracy: 1.0000
Epoch 7/20
46/46 [==============================] - 27s 577ms/step - loss: 0.0416 - accuracy: 1.0000 - val_loss: 0.0358 - val_accuracy: 1.0000

In [ ]:
#Classification report for testing
test_predIdxs = vgg16_model.predict_generator(test_generator)
test_predIdxs = np.argmax(test_predIdxs, axis=1)
print(classification_report(test_generator.labels, test_predIdxs, target_names=[l for l in labels]))

<ipython-input-8-52463c944575>:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  test_predIdxs = vgg16_model.predict_generator(test_generator)


              precision    recall  f1-score   support

        1001       1.00      1.00      1.00         4
        1003       1.00      1.00      1.00         6
        1004       1.00      1.00      1.00         6
        1002       1.00      1.00      1.00        10
        1006       1.00      1.00      1.00         6
        1005       1.00      1.00      1.00         5
        1008       1.00      1.00      1.00         5
        1009       1.00      1.00      1.00         7
        1007       1.00      1.00      1.00         8
        1015       1.00      1.00      1.00         6
        1012       1.00      1.00      1.00         8
        1010       1.00      1.00      1.00         6
        1014       1.00      1.00      1.00         5
        1011       1.00      1.00      1.00         5
        1016       1.00      1.00      1.00         5
        1017       1.00      1.00      1.00         7
        1013       1.00      1.00      1.00         5
        1022       1.00    